# New Hospitals Model

This notebook runs the NHP model and produces the aggregated results.

In [1]:
params_file = "sample_params.json"

## Setup

Load the required packages

In [2]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import numpy as np

import os
import shutil
import json
from datetime import datetime

from model.aae import AaEModel
from model.inpatients import InpatientsModel
from model.outpatients import OutpatientsModel

We need to create a folder ready to store the results.

In [3]:
with open(params_file, "r", encoding="UTF-8") as pf:
  params = json.load(pf)

dataset = params["input_data"]
scenario = params["name"]
model_runs = params["model_runs"]

run_time = datetime.now().strftime("%Y%m%d_%H%M%S")

results_path = f"data/{dataset}/results/{scenario}/{run_time}"

os.makedirs(results_path)
shutil.copy(params_file, f"{results_path}/params.json")

'data/RL4/results/test/20220407_120132/params.json'

We will run the model in parallel. By default, use all available CPU cores. You can set this to a lower value to use less resources, but it will take longer to run the model.

In [4]:
cpus = os.cpu_count()
cpus

12

When we run the model in parallel it's slightly more efficient to run a batch of model runs. Batches of 4 or 8 seems to be most efficient. This value should be a power of 2.

In [5]:
batch_size = 2 ** 2
batch_size

4

## Run the model

In [6]:
(AaEModel(results_path)
# add 1 because of the principal run
  .multi_model_runs(0, model_runs + 1, cpus, batch_size)
)

260it [00:13, 19.48it/s]                         

Model runs completed: 257 / 257


In [7]:
(OutpatientsModel(results_path)
  .multi_model_runs(0, model_runs + 1, cpus, batch_size)
)

260it [00:18, 13.97it/s]                         

Model runs completed: 257 / 257


In [8]:
(InpatientsModel(results_path)
  .multi_model_runs(0, model_runs + 1, cpus, batch_size)
)

KeyError: 'admission_avoidance_strategy'